# Semantic Segmentation Using Preprocessed TFRecords Dataset

In [ ]:
import os
import sys
import cv2
import PIL
import time
import sklearn
import numpy as np
import matplotlib.pyplot as plt
from IPython import display
from IPython.display import clear_output
from skimage.transform import resize

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.losses import *
from tensorflow.keras import backend as K
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import optimizers, models
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import array_to_img, img_to_array, load_img
from tensorflow.keras.mixed_precision import experimental as mixed_precision
K.clear_session()

physical_devices = tf.config.experimental.list_physical_devices("GPU")
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

# Automatic mixed precision gives around a 2X train speed-up and allows for 2X larger batch size. 
# Requires final layer be manually cast to float32
def enable_amp():
    # tf.config.optimizer.set_jit(True)
    policy = mixed_precision.Policy("mixed_float16")
    mixed_precision.set_policy(policy)
    
print("Tensorflow version: ", tf.__version__)
print(physical_devices)
enable_amp()

In [ ]:
def read_tfrecord(serialized_example):
    feature_description = {
        'image': tf.io.FixedLenFeature((), tf.string),
        'segmentation': tf.io.FixedLenFeature((), tf.string),
        'height': tf.io.FixedLenFeature((), tf.int64),
        'width': tf.io.FixedLenFeature((), tf.int64),
        'image_depth': tf.io.FixedLenFeature((), tf.int64),
        'mask_depth': tf.io.FixedLenFeature((), tf.int64),
    }
    example = tf.io.parse_single_example(serialized_example, feature_description)
    
    image = tf.io.parse_tensor(example['image'], out_type = float)
    image_shape = [example['height'], example['width'], 3]
    image = tf.reshape(image, image_shape)
    
    mask = tf.io.parse_tensor(example['segmentation'], out_type = tf.uint8)
    mask_shape = [example['height'], example['width'], 1]
    mask = tf.reshape(mask, mask_shape)
    
    return image, mask


def get_dataset_from_tfrecord(tfrecord_dir):
    tfrecord_dataset = tf.data.TFRecordDataset(tfrecord_dir)
    parsed_dataset = tfrecord_dataset.map(read_tfrecord)
    return parsed_dataset

In [ ]:
train_tfrecord_dir = 'oxford_pets\\train.tfrecords'
test_tfrecord_dir = 'oxford_pets\\test.tfrecords'

img_height = 384
img_width = 384
n_classes = 4

In [ ]:
@tf.function
def mask_to_categorical(image, mask):
    mask = tf.squeeze(mask)
    mask = tf.one_hot(tf.cast(mask, tf.int32), n_classes)
    mask = tf.cast(mask, tf.float32)
    return image, mask


@tf.function
def load_image_train(input_image, input_mask):
    input_image = tf.image.resize(input_image, (img_height, img_width))
    input_mask = tf.image.resize(input_mask, (img_height, img_width))

    if tf.random.uniform(()) > 0.5:
        input_image = tf.image.flip_left_right(input_image)
        input_mask = tf.image.flip_left_right(input_mask)

    input_image, input_mask = mask_to_categorical(input_image, input_mask)
    input_mask = tf.squeeze(input_mask)

    return input_image, input_mask

def load_image_test(input_image, input_mask):
    input_image = tf.image.resize(input_image, (img_height, img_width))
    input_mask = tf.image.resize(input_mask, (img_height, img_width))
    
    input_image, input_mask = mask_to_categorical(input_image, input_mask)
    input_mask = tf.squeeze(input_mask)

    return input_image, input_mask

In [ ]:
TRAIN_LENGTH = 5912
TEST_LENGTH = 1478
BATCH_SIZE = 8
BUFFER_SIZE = TRAIN_LENGTH 
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE

In [ ]:
train_tfrecords_dataset = get_dataset_from_tfrecord(train_tfrecord_dir)
test_tfrecords_dataset = get_dataset_from_tfrecord(test_tfrecord_dir)

In [ ]:
# Preprocessing: resize the images and masks, flip them, and normalize them
train = train_tfrecords_dataset.map(load_image_train, num_parallel_calls=tf.data.experimental.AUTOTUNE)
test = test_tfrecords_dataset.map(load_image_test)

In [ ]:
# the cache() transformation reduces resource usage
train_dataset = train.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
test_dataset = test.batch(BATCH_SIZE)

In [ ]:
def display(display_list):
    plt.figure(figsize=(15, 5))
    title = ['Input Image', 'True Mask', 'Predicted Mask']
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
for image, mask in test.take(13):
    sample_image, sample_mask = image, mask
    
sample_mask = tf.argmax(sample_mask, axis=-1)
sample_mask = sample_mask[..., tf.newaxis]
display([sample_image, sample_mask])

### Define the model

In [ ]:
def unet_model(input_height=img_height,  input_width=img_width, n_classes = 3):
    
    img_input = tf.keras.layers.Input(shape=(input_height, input_width, 3))

    # -------------------------- Encoder --------------------------
    
    c1 = Conv2D(64, 3, padding='same', activation="selu")(img_input)
    c1 = Conv2D(64, 3, padding='same', activation="selu")(c1)
    p1 = MaxPooling2D((2,2))(c1)
    
    c2 = Conv2D(128, 3, padding='same', activation="selu")(p1)
    c2 = Conv2D(128, 3, padding='same', activation="selu")(c2)
    p2 = MaxPooling2D((2,2))(c2)
    p2 = Dropout(0.1)(p2)
    
    c3 = Conv2D(256, 3, padding='same', activation="selu")(p2)
    c3 = Conv2D(256, 3, padding='same', activation="selu")(c3)
    p3 = MaxPooling2D((2,2))(c3)
    p3 = Dropout(0.2)(p3)
    
    c4 = Conv2D(512, 3, padding='same', activation="selu")(p3)
    c4 = Conv2D(512, 3, padding='same', activation="selu")(c4)
    p4 = MaxPooling2D((2,2))(c4)
    p4 = Dropout(0.3)(p4)
    
    # ------------------------ Bottleneck -------------------------
    
    c5 = Conv2D(1024, 3, padding='same', activation="selu")(p4)
    c5 = Conv2D(1024, 3, padding='same', activation="selu")(c5)
    c5 = Dropout(0.5)(c5)
    
    # -------------------------- Decoder --------------------------
    # Can also use transpose convolutions instead of upsampling
    
    u6 = concatenate([UpSampling2D(2)(c5), c4])
    c6 = Conv2D(512, 3, padding='same')(u6)
    c6 = BatchNormalization()(c6)
    c6 = Activation('selu')(c6)
    c6 = Conv2D(256, 3, padding='same')(c6)
    c6 = BatchNormalization()(c6)
    c6 = Activation('selu')(c6)
    c6 = Dropout(0.3)(c6)
    
    u7 = concatenate([UpSampling2D(2)(c6), c3])
    c7 = Conv2D(256, 3, padding='same')(u7)
    c7 = BatchNormalization()(c7)
    c7 = Activation('selu')(c7)
    c7 = Conv2D(128, 3, padding='same')(c7)
    c7 = BatchNormalization()(c7)
    c7 = Activation('selu')(c7)
    c7 = Dropout(0.2)(c7)

    u8 = concatenate([UpSampling2D(2)(c7), c2])
    c8 = Conv2D(128, 3, padding='same')(u8)
    c8 = BatchNormalization()(c8)
    c8 = Activation('selu')(c8)
    c8 = Conv2D(64, 3, padding='same')(c8)
    c8 = BatchNormalization()(c8)
    c8 = Activation('selu')(c8)
    c8 = Dropout(0.1)(c8)

    u9 = concatenate([UpSampling2D(2)(c8), c1]) 
    c9 = Conv2D(64, 3, padding='same')(u9)
    c9 = BatchNormalization()(c9)
    c9 = Activation('selu')(c9)
    c9 = Conv2D(64, 3, padding='same')(u9)
    c9 = BatchNormalization()(c9)
    c9 = Activation('selu')(c9)
    c9 = Conv2D(n_classes, 3, padding='same')(c9)
    
    # Softmax for multi-class prediction
    output = Activation("softmax", dtype='float32')(c9)
    
    return tf.keras.Model(inputs=img_input, outputs=output)   



def xception_model(input_height=img_height,  input_width=img_width, n_classes = 4):
    
    inputs = tf.keras.layers.Input(shape=(input_height, input_width, 3))

    ######## [First half of the network: downsampling inputs] ########

    # Entry block
    x = Conv2D(64, 3, strides=2, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("selu")(x)

    previous_block_activation = x  # Set aside residual

    # Blocks 1, 2, 3 are identical apart from the feature depth.
    for filters in [128, 256, 512]:
        x = Activation("selu")(x)
        x = SeparableConv2D(filters, 3, padding="same")(x)
        x = BatchNormalization()(x)

        x = Activation("selu")(x)
        x = SeparableConv2D(filters, 3, padding="same")(x)
        x = BatchNormalization()(x)

        x = MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = Conv2D(filters, 1, strides=2, padding="same")(previous_block_activation)
        x = add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    ######## [Second half of the network: upsampling inputs] ########

    previous_block_activation = x  # Set aside residual

    for filters in [512, 256, 128, 64]:
        x = Activation("selu")(x)
        x = Conv2DTranspose(filters, 3, padding="same")(x)
        x = BatchNormalization()(x)

        x = Activation("selu")(x)
        x = Conv2DTranspose(filters, 3, padding="same")(x)
        x = BatchNormalization()(x)

        x = UpSampling2D(2)(x)

        # Project residual
        residual = UpSampling2D(2)(previous_block_activation)
        residual = Conv2D(filters, 1, padding="same")(residual)
        x = add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    # Add a per-pixel classification layer
    x = Conv2D(n_classes, 3, padding="same")(x)
    
    outputs = Activation("softmax", dtype='float32')(x)

    # Define the model
    model = tf.keras.Model(inputs, outputs)
    return model

In [ ]:
def dice_coef(y_true, y_pred):
    """Pixel-wise dice coefficient calculated for each class and averaged over batch. Final dice score is the average 
    of the dice scores for each class."""
    dice = 0.0
    smooth = 1.0
    for i in range(1, n_classes):
        intersection = y_true[:,:,i] * y_pred[:,:,i]
        all_ = y_true[:,:,i] + y_pred[:,:,i]
        intersection = K.sum(intersection, 1)
        all_ = K.sum(all_, 1)
        temp = (2. * intersection + smooth) / (all_ + smooth)
        temp = K.mean(temp)
        dice = dice + temp
    return dice / (n_classes-1)


# Custom loss function combines categorical cross entropy and pixel-wise dice coefficient. Dice loss is especially good for 
# datasets with class-imbalance
def cce_dice_loss(y_true, y_pred):
    return (tf.keras.losses.categorical_crossentropy(y_true, y_pred) - dice_coef(y_true, y_pred)) + 1

In [ ]:
model = unet_model(input_height=img_height, input_width=img_width, n_classes=4)
# model = xception_model(input_height=img_height, input_width=img_width, n_classes=4)

In [ ]:
def create_mask(pred_mask):
    pred_mask = tf.squeeze(pred_mask, axis=0)
    pred_mask = tf.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    return pred_mask


def show_predictions():
    pred_mask = model.predict(sample_image[tf.newaxis, ...])
    display([sample_image, sample_mask, create_mask(pred_mask)])

        
class DisplayCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        clear_output(wait=True)
        show_predictions()
        print ('\nSample Prediction after epoch {}\n'.format(epoch+1))
        
show_predictions()

In [ ]:
model_name = "saved_models\\unet_pets.h5"

model.compile(optimizer = Adam(lr=1e-4),
              loss = cce_dice_loss, 
              metrics = ['accuracy', dice_coef])

callbacks = [
    DisplayCallback(),
    EarlyStopping(monitor='val_loss', mode='min', patience=9, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', mode='min', patience=3, factor=0.1, min_lr=1e-10, verbose=1),
    ModelCheckpoint(model_name, monitor='val_loss', verbose=1, mode='min', save_best_only=True, save_weights_only=True)
]
# model.load_weights("big_unet_model.h5", by_name=True)

In [ ]:
EPOCHS = 100
STEPS_PER_EPOCH = TRAIN_LENGTH // BATCH_SIZE
VALIDATION_STEPS = TEST_LENGTH//BATCH_SIZE

In [ ]:
start_time = time.time()
results = model.fit(train_dataset,
                    steps_per_epoch=STEPS_PER_EPOCH,
                    validation_steps=VALIDATION_STEPS,
                    epochs = EPOCHS,
                    validation_data = test_dataset,
                    callbacks = callbacks,
                    verbose = 1)
end_time = time.time()

t_minutes = (end_time - start_time) // 60
t_per_epoch = (end_time - start_time)  // len(results.history["loss"])
print("#### time: {} min at {} s per epoch".format(t_minutes, t_per_epoch))

In [ ]:
# model.load_weights("saved_models\\unet_pets.h5")

In [ ]:
plt.figure(figsize=(15,7))

plt.subplot(1,3,1)  
plt.plot(results.history['loss'], 'r', label='Training loss')
plt.plot(results.history['val_loss'], 'b', label='Validation loss')
plt.title('Log Loss', fontsize=16)
plt.xlabel('Epoch', fontsize=16)
plt.ylabel('log_loss', fontsize=16)
plt.ylim([0, 1])
plt.legend()

plt.subplot(1,3,2)
plt.plot(results.history['accuracy'], 'r', label='Training accuracy')
plt.plot(results.history['val_accuracy'], 'b', label='Validation accuracy')
plt.title('Accuracy', fontsize=16)
plt.xlabel('Epoch', fontsize=16)
plt.ylabel('Accuracy', fontsize=16)
plt.ylim([0, 1])
plt.legend()

plt.subplot(1,3,3)
plt.plot(results.history['dice_coef'], 'r', label='Dice coefficient')
plt.plot(results.history['val_dice_coef'], 'b', label='Validation dice coefficient')
plt.title('Dice Coefficient', fontsize=16)
plt.xlabel('Epoch', fontsize=16)
plt.ylabel('Dice', fontsize=16)
plt.ylim([0, 1])
plt.legend()
plt.show()

In [ ]:
n_samples = 500

X_test = np.zeros((n_samples, img_height, img_width, 3))
y_test = np.zeros((n_samples, img_height, img_width, 4))

for idx, (image, mask) in enumerate(test):
    X_test[idx] = image.numpy()
    y_test[idx] = mask.numpy()
    if idx == (n_samples-1):
        break

print("X_test.shape: {} , y_test.shape: {}".format(X_test.shape, y_test.shape))

In [ ]:
img_num = 38
sample_image = X_test[img_num]
sample_mask = np.expand_dims(np.argmax(y_test[img_num], axis=-1), axis=-1)

def create_mask(pred_mask):
    pred_mask = tf.squeeze(pred_mask, axis=0)
    pred_mask = tf.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    return pred_mask


def show_predictions():
    pred_mask = model.predict(sample_image[tf.newaxis, ...])
    display([sample_image, sample_mask, create_mask(pred_mask)])
    
show_predictions()

In [ ]:
y_pred = model.predict(X_test[0:n_samples])

In [ ]:
# Quick way to evaluate model using dice coefficient evaluated over larger set of datapoints
def dice_coef_eval(y_true, y_pred):
    dice = 0.0
    smooth = 1.0
    class_dice = []
    for i in range(1, n_classes):
        intersection = y_true[:,:,i] * y_pred[:,:,i]
        all_ = y_true[:,:,i] + y_pred[:,:,i]
        intersection = K.sum(intersection, 1)
        all_ = K.sum(all_, 1)
        temp = (2. * intersection + smooth) / (all_ + smooth)
        temp = K.mean(temp)
        class_dice.append(round(temp.numpy(), 4))
        dice = dice + temp
    total_dice = dice / (n_classes-1)
    return class_dice, round(total_dice.numpy(), 4)

In [ ]:
class_dice, total_dice = dice_coef_eval(y_test[0:n_samples], y_pred)

In [ ]:
print("class 1: {} \nclass 2: {} \nclass 3: {} \ntotal: {}".format(class_dice[0], class_dice[1], class_dice[2], total_dice))

In [ ]:
# 89.16
# 89.14
# 87.18
# 90.13